In [50]:
import pandas as pd
import numpy as np
import scipy as sci
from scipy import signal
import matplotlib.pyplot as plt
from sklearn import model_selection
import seaborn as sns
import neurokit2 as nk
import wfdb #library for reading ecg data https://github.com/MIT-LCP/wfdb-python/blob/main/demo.ipynb
import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button

In [51]:
#Datos con arrythmia
data = {
    
}

baseline_data = r"G:\Shared drives\Proyecto Final\Progra\databases\MIT Arrythmia\100"

fs = 360
time = 10  #se utilizara desde el segundo 10 para eviar el ruido inicial de los primeros readings
sample_from = time * fs
sample_to = 20*fs # 5s de sampleo

full_record = record = wfdb.rdrecord(baseline_data)
full_record_data = full_record.__dict__
print(full_record_data)
n_samples = full_record_data['sig_len']
print(n_samples)
try:
  ecg, info = nk.ecg_process(full_record.to_dataframe()['MLII'], sampling_rate = fs)
except:
  ecg, info = nk.ecg_process(full_record.to_dataframe()['V2'], sampling_rate = fs)

nk.ecg_plot(ecg, sampling_rate=fs)


x= True
while x == True:
  try:
    sample_from = time * fs
    sample_to = (time+10)*fs # 10s de sampleo
    data[f"{sample_from}-{sample_to}"] = {}
    data[f"{sample_from}-{sample_to}"][f"raw_data"] = record = wfdb.rdrecord(baseline_data, sampfrom = sample_from, sampto = sample_to)
  except:
    data.pop(f"{sample_from}-{sample_to}") 
  try:
    data[f"{sample_from}-{sample_to}"]["ecg_signals"], data[f"{sample_from}-{sample_to}"]["info"] = nk.ecg_process(data[f"{sample_from}-{sample_to}"]["raw_data"].to_dataframe()["MLII"], sampling_rate=fs) 
  except:
    data[f"{sample_from}-{sample_to}"]["ecg_signals"], data[f"{sample_from}-{sample_to}"]["info"] = nk.ecg_process(data[f"{sample_from}-{sample_to}"]["raw_data"].to_dataframe()["V2"], sampling_rate=fs) 
  time+=10
  if (time+10)*fs > n_samples*fs:
    print("hey!")
    x = False

data

{'record_name': '100', 'n_sig': 2, 'fs': 360, 'counter_freq': None, 'base_counter': None, 'sig_len': 650000, 'base_time': None, 'base_date': None, 'comments': ['69 M 1085 1629 x1', 'Aldomet, Inderal'], 'sig_name': ['MLII', 'V5'], 'p_signal': array([[-0.145, -0.065],
       [-0.145, -0.065],
       [-0.145, -0.065],
       ...,
       [-0.675, -0.365],
       [-0.765, -0.335],
       [-1.28 ,  0.   ]]), 'd_signal': None, 'e_p_signal': None, 'e_d_signal': None, 'file_name': ['100.dat', '100.dat'], 'fmt': ['212', '212'], 'samps_per_frame': [1, 1], 'skew': [None, None], 'byte_offset': [None, None], 'adc_gain': [200.0, 200.0], 'baseline': [1024, 1024], 'units': ['mV', 'mV'], 'adc_res': [11, 11], 'adc_zero': [1024, 1024], 'init_value': [995, 1011], 'checksum': [-22131, 20052], 'block_size': [0, 0]}
650000


KeyError: 'V2'

In [ ]:
ecg["ECG_Raw"].plot()

In [ ]:
#Para cargar datos sin arrythmia
data = {
    
}

baseline_data = r"G:\Shared drives\Proyecto Final\Progra\databases\PTB\patient116\s0302lre"

fs = 1000
time = 10  #se utilizara desde el segundo 10 para eviar el ruido inicial de los primeros readings
sample_from = time * fs
sample_to = 20*fs # 5s de sampleo

full_record = record = wfdb.rdrecord(baseline_data, sampfrom = sample_from, sampto = sample_to)
full_record_data = full_record.__dict__
print(full_record_data)
n_samples = full_record_data['sig_len']
print(n_samples)
ecg, info = nk.ecg_process(full_record.to_dataframe()['ii'], sampling_rate = fs)
nk.ecg_plot(ecg, sampling_rate=fs)


x= True
while x == True:
  try:
    sample_from = time * fs
    sample_to = (time+10)*fs # 10s de sampleo
    data[f"{sample_from}-{sample_to}"] = {}
    data[f"{sample_from}-{sample_to}"][f"raw_data"] = record = wfdb.rdrecord(baseline_data, sampfrom = sample_from, sampto = sample_to)
  except:
    data.pop(f"{sample_from}-{sample_to}") 
  try:
    data[f"{sample_from}-{sample_to}"]["ecg_signals"], data[f"{sample_from}-{sample_to}"]["info"] = nk.ecg_process(data[f"{sample_from}-{sample_to}"]["raw_data"].to_dataframe()["ii"], sampling_rate=fs) 
  except:
    pass
  time+=10
  if (time+10)*fs > n_samples*fs:
    print("hey!")
    x = False

data


In [ ]:
for item in data:
  for feature in data[item]['ecg_signals']:
    data[item][feature] = data[item]['ecg_signals'][feature]

In [ ]:
data_df = pd.DataFrame.from_dict(data, orient= 'index')
data_df

,raw_data,ecg_signals,info,ECG_Raw,ECG_Clean,ECG_Rate,ECG_Quality,ECG_R_Peaks,ECG_P_Peaks,ECG_P_Onsets,...,ECG_R_Onsets,ECG_R_Offsets,ECG_S_Peaks,ECG_T_Peaks,ECG_T_Onsets,ECG_T_Offsets,ECG_Phase_Atrial,ECG_Phase_Completion_Atrial,ECG_Phase_Ventricular,ECG_Phase_Completion_Ventricular
3600-7200,<wfdb.io.record.Record object at 0x000001F9457...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [262, 570, 865, 1164, 1460, 17...",0 -0.390 1 -0.395 2 -0.390 3 ...,0 -0.026905 1 -0.028182 2 -0.02...,0 73.242910 1 73.242910 2 73...,0 0.486675 1 0.486675 2 0.48...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 NaN 1 NaN 2 NaN 3 NaN 4 ...,0 0.0 1 0.0 2 0.0 3 0....,0 NaN 1 NaN 2 NaN 3 NaN 4 ...,0 0.0 1 0.0 2 0.0 3 0....
7200-10800,<wfdb.io.record.Record object at 0x000001F90F8...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [191, 469, 753, 1045, 1339, 16...",0 -0.405 1 -0.410 2 -0.410 3 ...,0 -0.009828 1 -0.009593 2 -0.00...,0 74.250000 1 74.250000 2 74...,0 0.747294 1 0.747294 2 0.74...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 NaN 1 NaN 2 NaN 3 NaN 4 ...,0 0.0 1 0.0 2 0.0 3 0....,0 NaN 1 NaN 2 NaN 3 NaN 4 ...,0 0.0 1 0.0 2 0.0 3 0....
10800-14400,<wfdb.io.record.Record object at 0x000001F90F8...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [391, 680, 980, 1266, 1550, 18...",0 -0.385 1 -0.365 2 -0.375 3 ...,0 0.014942 1 0.016895 2 0.01...,0 74.787535 1 74.787535 2 74...,0 0.298503 1 0.298503 2 0.29...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 NaN 1 NaN 2 NaN 3 NaN 4 ...,0 0.0 1 0.0 2 0.0 3 0....,0 NaN 1 NaN 2 NaN 3 NaN 4 ...,0 0.0 1 0.0 2 0.0 3 0....
14400-18000,<wfdb.io.record.Record object at 0x000001F90F8...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [310, 611, 910, 1207, 1499, 17...",0 -0.360 1 -0.360 2 -0.360 3 ...,0 0.088598 1 0.084410 2 0.08...,0 73.423980 1 73.423980 2 73...,0 0.0 1 0.0 2 0.0 3 0....,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 NaN 1 NaN 2 NaN 3 NaN 4 ...,0 0.0 1 0.0 2 0.0 3 0....,0 NaN 1 NaN 2 NaN 3 NaN 4 ...,0 0.0 1 0.0 2 0.0 3 0....
18000-21600,<wfdb.io.record.Record object at 0x000001F90F8...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [227, 514, 795, 1080, 1388, 16...",0 -0.450 1 -0.445 2 -0.450 3 ...,0 -0.030782 1 -0.030008 2 -0.02...,0 74.342929 1 74.342929 2 74...,0 0.933698 1 0.933698 2 0.93...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 NaN 1 NaN 2 NaN 3 NaN 4 ...,0 0.0 1 0.0 2 0.0 3 0....,0 NaN 1 NaN 2 NaN 3 NaN 4 ...,0 0.0 1 0.0 2 0.0 3 0....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630000-633600,<wfdb.io.record.Record object at 0x000001F90F5...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [289, 561, 837, 1110, 1377, 16...",0 -0.355 1 -0.360 2 -0.360 3 ...,0 -0.080307 1 -0.078844 2 -0.07...,0 80.772826 1 80.772826 2 80...,0 0.861495 1 0.861495 2 0.86...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 0 1 0 2 0 3 0 4 ...,0 NaN 1 NaN 2 NaN 3 NaN 4 ...,0 0.0 1 0.0 2 0.0 3 0....,0 NaN 1 NaN 2 NaN 3 NaN 4 ...,0 0.0 1 0.0 2 0.0 3 0....
633600-637200,<wfdb.io.record.Record obj

In [ ]:
ecg_signal_data = ['ECG_P_Peaks', 'ECG_Q_Peaks', 'ECG_S_Peaks', 'ECG_T_Peaks', 'ECG_R_Peaks']

def dataAdquisition(data,fs):
  for item in data:
    peaks, info = nk.ecg_peaks(data[item]["ecg_signals"]["ECG_Clean"], sampling_rate=fs)
    # data[item]["r_peaks"] = data[item][e]
    data[item]["hrv_time"] = nk.hrv_time(peaks, sampling_rate=fs, show=False)
    for var in data[item]["hrv_time"]:
      data[item][var] = data[item]["hrv_time"][var].to_frame().iloc[0][var]
    for feature in ecg_signal_data:
      data[item][feature] = data[item]["ecg_signals"][feature].sum()
    data[item]["mean_quality"]  = data[item]["ecg_signals"]["ECG_Quality"].mean()


  return data

In [ ]:
control_data_clean = dataAdquisition(data, fs)

In [ ]:
#sin arrythmia
# for item in control_data_clean:
#   dict_data = control_data_clean[item]["raw_data"].__dict__
#   control_data_clean[item]["sex"] = dict_data['comments'][1].replace('sex: ', '')
#   control_data_clean[item]["age"] = dict_data['comments'][0].replace('age: ', '')

In [ ]:
# #sin arrythmia
# control_data_df = pd.DataFrame.from_dict(control_data_clean, orient = 'index')
# control_data_df = control_data_df.join(pd.get_dummies(control_data_df["sex"])).drop("sex", axis = 1)

In [ ]:
#Revisar el codigo sin esto

#arrythmia
# for item in control_data_clean:
#   dict_data = control_data_clean[item]["raw_data"].__dict__
#   control_data_clean[item]["sex"] = dict_data['comments'][0].split(' ')[1]
#   control_data_clean[item]["age"] = dict_data['comments'][0].split(' ')[0]

control_data_df = pd.DataFrame.from_dict(control_data_clean, orient = 'index')
# control_data_df = control_data_df.join(pd.get_dummies(control_data_df["sex"])).drop("sex", axis = 1)

In [ ]:
control_data_df

,raw_data,ecg_signals,info,ECG_Raw,ECG_Clean,ECG_Rate,ECG_Quality,ECG_R_Peaks,ECG_P_Peaks,ECG_P_Onsets,...,HRV_IQRNN,HRV_Prc20NN,HRV_Prc80NN,HRV_pNN50,HRV_pNN20,HRV_MinNN,HRV_MaxNN,HRV_HTI,HRV_TINN,mean_quality
3600-7200,<wfdb.io.record.Record object at 0x000001F9457...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [262, 570, 865, 1164, 1460, 17...",0 -0.390 1 -0.395 2 -0.390 3 ...,0 -0.026905 1 -0.028182 2 -0.02...,0 73.242910 1 73.242910 2 73...,0 0.486675 1 0.486675 2 0.48...,12,12,0 0 1 0 2 0 3 0 4 ...,...,30.555556,794.444444,830.555556,0.000000,54.545455,786.111111,869.444444,3.666667,46.8750,0.713610
7200-10800,<wfdb.io.record.Record object at 0x000001F90F8...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [191, 469, 753, 1045, 1339, 16...",0 -0.405 1 -0.410 2 -0.410 3 ...,0 -0.009828 1 -0.009593 2 -0.00...,0 74.250000 1 74.250000 2 74...,0 0.747294 1 0.747294 2 0.74...,12,12,0 0 1 0 2 0 3 0 4 ...,...,33.333333,788.888889,827.777778,9.090909,45.454545,772.222222,858.333333,5.500000,15.6250,0.536724
10800-14400,<wfdb.io.record.Record object at 0x000001F90F8...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [391, 680, 980, 1266, 1550, 18...",0 -0.385 1 -0.365 2 -0.375 3 ...,0 0.014942 1 0.016895 2 0.01...,0 74.787535 1 74.787535 2 74...,0 0.298503 1 0.298503 2 0.29...,12,12,0 0 1 0 2 0 3 0 4 ...,...,36.111111,788.888889,833.333333,18.181818,81.818182,658.333333,880.555556,5.500000,140.6250,0.491688
14400-18000,<wfdb.io.record.Record object at 0x000001F90F8...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [310, 611, 910, 1207, 1499, 17...",0 -0.360 1 -0.360 2 -0.360 3 ...,0 0.088598 1 0.084410 2 0.08...,0 73.423980 1 73.423980 2 73...,0 0.0 1 0.0 2 0.0 3 0....,12,12,0 0 1 0 2 0 3 0 4 ...,...,26.388889,802.777778,833.333333,0.000000,36.363636,780.555556,841.666667,3.666667,54.6875,0.618860
18000-21600,<wfdb.io.record.Record object at 0x000001F90F8...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [227, 514, 795, 1080, 1388, 16...",0 -0.450 1 -0.445 2 -0.450 3 ...,0 -0.030782 1 -0.030008 2 -0.02...,0 74.342929 1 74.342929 2 74...,0 0.933698 1 0.933698 2 0.93...,12,12,0 0 1 0 2 0 3 0 4 ...,...,33.333333,786.111111,822.222222,9.090909,36.363636,780.555556,855.555556,3.666667,0.0000,0.519613
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
630000-633600,<wfdb.io.record.Record object at 0x000001F90F5...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [289, 561, 837, 1110, 1377, 16...",0 -0.355 1 -0.360 2 -0.360 3 ...,0 -0.080307 1 -0.078844 2 -0.07...,0 80.772826 1 80.772826 2 80...,0 0.861495 1 0.861495 2 0.86...,13,13,0 0 1 0 2 0 3 0 4 ...,...,29.166667,719.444444,757.777778,0.000000,16.666667,716.666667,766.666667,4.000000,0.0000,0.618084
633600-637200,<wfdb.io.record.Record object at 0x000001F90F5...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [168, 428, 683, 945, 1226, 150...",0 -0.385 1 -0.385 2 -0.370 3 ...,0 0.007044 1 0.015577 2 0.02...,0 77.674558 1 77.674558 2 77...,0 0.512603 1 0.512603 2 0.51...,13,13,0 0 1 0 2 0 3 0 4 ...,...,50.694444,733.333333,803.888889,8.333333,41.666667,708.333333,816.666667,6.000000,78.1250,0.545999
637200-640800,<wfdb.io.record.Record object at 0x000001F90F5...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [199, 504, 797, 1082, 1360, 16...",0 -0.370 1 -0.365 2 -0.370 3 ...,0 -0.007626 1 -0.007604 2 -0.00...,0 73.880597 1 73.880597 2 73...,0 0.809723 1 0.809723 2 0.80...,12,12,0 0 1 0 2 0 3 0 4 ...,...,29.166667,794.444444,827.777778,0.000000,54.545455,772.222222,850.000000,3.666667,23.4375,0.516092
640800-644400,<wfdb.io.record.Record object at 0x000001F90F6...,ECG_Raw ECG_Clean ECG_Rate ECG_Qual...,"{'ECG_R_Peaks': [377, 680, 977, 1265, 1537, 18...",0 -0.365 1 -0.355 2 -0.350 3 ...,0 -0.027943 1 -0.024194 2 -0.02...,0 76.423287 1 76.423287 2 76...,0 1.110223e-16 1 1.110223e-16 2 ...,12,12,0 0 1 0 2 0 3 0 4 ...,...,27.777778,763.888889,800.000000,0.000000,18.181818,755

In [ ]:
control_data_df = control_data_df[control_data_df["mean_quality"]>0.5]

In [ ]:
control_data_df

,raw_data,ecg_signals,info,ECG_Raw,ECG_Clean,ECG_Rate,ECG_Quality,ECG_R_Peaks,ECG_P_Peaks,ECG_P_Onsets,...,HRV_IQRNN,HRV_Prc20NN,HRV_Prc80NN,HRV_pNN50,HRV_pNN20,HRV_MinNN,HRV_MaxNN,HRV_HTI,HRV_TINN,mean_quality
3600-7200,<wfdb.io.record.Record object at 0x000001F9457...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [262, 570, 865, 1164, 1460, 17...",0 -0.390 1 -0.395 2 -0.390 3 ...,0 -0.026905 1 -0.028182 2 -0.02...,0 73.242910 1 73.242910 2 73...,0 0.486675 1 0.486675 2 0.48...,12,12,0 0 1 0 2 0 3 0 4 ...,...,30.555556,794.444444,830.555556,0.000000,54.545455,786.111111,869.444444,3.666667,46.8750,0.713610
7200-10800,<wfdb.io.record.Record object at 0x000001F90F8...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [191, 469, 753, 1045, 1339, 16...",0 -0.405 1 -0.410 2 -0.410 3 ...,0 -0.009828 1 -0.009593 2 -0.00...,0 74.250000 1 74.250000 2 74...,0 0.747294 1 0.747294 2 0.74...,12,12,0 0 1 0 2 0 3 0 4 ...,...,33.333333,788.888889,827.777778,9.090909,45.454545,772.222222,858.333333,5.500000,15.6250,0.536724
14400-18000,<wfdb.io.record.Record object at 0x000001F90F8...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [310, 611, 910, 1207, 1499, 17...",0 -0.360 1 -0.360 2 -0.360 3 ...,0 0.088598 1 0.084410 2 0.08...,0 73.423980 1 73.423980 2 73...,0 0.0 1 0.0 2 0.0 3 0....,12,12,0 0 1 0 2 0 3 0 4 ...,...,26.388889,802.777778,833.333333,0.000000,36.363636,780.555556,841.666667,3.666667,54.6875,0.618860
18000-21600,<wfdb.io.record.Record object at 0x000001F90F8...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [227, 514, 795, 1080, 1388, 16...",0 -0.450 1 -0.445 2 -0.450 3 ...,0 -0.030782 1 -0.030008 2 -0.02...,0 74.342929 1 74.342929 2 74...,0 0.933698 1 0.933698 2 0.93...,12,12,0 0 1 0 2 0 3 0 4 ...,...,33.333333,786.111111,822.222222,9.090909,36.363636,780.555556,855.555556,3.666667,0.0000,0.519613
21600-25200,<wfdb.io.record.Record object at 0x000001F90F8...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [128, 429, 721, 1002, 1281, 15...",0 -0.235 1 -0.225 2 -0.235 3 ...,0 -0.014977 1 -0.016091 2 -0.01...,0 74.599686 1 74.599686 2 74...,0 0.963312 1 0.963312 2 0.96...,12,12,0 0 1 0 2 0 3 0 4 ...,...,36.111111,780.555556,825.000000,0.000000,54.545455,775.000000,841.666667,3.666667,0.0000,0.696082
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626400-630000,<wfdb.io.record.Record object at 0x000001F90F6...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [316, 600, 881, 1155, 1427, 17...",0 -0.450 1 -0.435 2 -0.440 3 ...,0 -0.121289 1 -0.115721 2 -0.11...,0 78.029557 1 78.029557 2 78...,0 0.609101 1 0.609101 2 0.60...,12,12,0 0 1 0 2 0 3 0 4 ...,...,48.611111,755.555556,813.888889,27.272727,45.454545,527.777778,900.000000,11.000000,0.0000,0.540232
630000-633600,<wfdb.io.record.Record object at 0x000001F90F5...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [289, 561, 837, 1110, 1377, 16...",0 -0.355 1 -0.360 2 -0.360 3 ...,0 -0.080307 1 -0.078844 2 -0.07...,0 80.772826 1 80.772826 2 80...,0 0.861495 1 0.861495 2 0.86...,13,13,0 0 1 0 2 0 3 0 4 ...,...,29.166667,719.444444,757.777778,0.000000,16.666667,716.666667,766.666667,4.000000,0.0000,0.618084
633600-637200,<wfdb.io.record.Record object at 0x000001F90F5...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [168, 428, 683, 945, 1226, 150...",0 -0.385 1 -0.385 2 -0.370 3 ...,0 0.007044 1 0.015577 2 0.02...,0 77.674558 1 77.674558 2 77...,0 0.512603 1 0.512603 2 0.51...,13,13,0 0 1 0 2 0 3 0 4 ...,...,50.694444,733.333333,803.888889,8.333333,41.666667,708.333333,816.666667,6.000000,78.1250,0.545999
637200-640800,<wfdb.io.record.Record object at 0x000001F90F5...,ECG_Raw ECG_Clean ECG_Rate ECG_Quali...,"{'ECG_R_Peaks': [199, 504, 797, 1082, 1360, 16...",0 -0.370 1 -0.365 2 -0.370 3 ...,0 -0.007626 1 -0.007604 2 -0.00...,0 73.880597 1 73.880597 2 73...,0 0.809723 1 0.809723 2 0.80...,12,12,0 0 1 0 2 0 3 0 4 ...,...,29.166667,794.444444,827.777778,0.000000,54.545455,772.2

In [ ]:
control_data_df.drop(columns = ['HRV_SDANN1', 'HRV_SDNNI1', 'HRV_SDANN2','HRV_SDNNI2', 'HRV_SDANN5', 'HRV_SDNNI5'], inplace = True)


C:\Users\reegi\AppData\Local\Temp\ipykernel_16632\1059642553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control_data_df.drop(columns = ['HRV_SDANN1', 'HRV_SDNNI1', 'HRV_SDANN2','HRV_SDNNI2', 'HRV_SDANN5', 'HRV_SDNNI5'], inplace = True)


In [ ]:
control_data_clean

{'3600-7200': {'raw_data': <wfdb.io.record.Record at 0x1f94576f130>,
  'ecg_signals':       ECG_Raw  ECG_Clean   ECG_Rate  ECG_Quality  ECG_R_Peaks  ECG_P_Peaks  \
  0      -0.390  -0.026905  73.242910     0.486675            0            0   
  1      -0.395  -0.028182  73.242910     0.486675            0            0   
  2      -0.390  -0.029561  73.242910     0.486675            0            0   
  3      -0.405  -0.030838  73.242910     0.486675            0            0   
  4      -0.405  -0.031299  73.242910     0.486675            0            0   
  ...       ...        ...        ...          ...          ...          ...   
  3595   -0.405   0.001557  76.325088     0.000000            0            0   
  3596   -0.410   0.001085  76.325088     0.000000            0            0   
  3597   -0.425   0.000528  76.325088     0.000000            0            0   
  3598   -0.430   0.000496  76.325088     0.000000            0            0   
  3599   -0.420   0.000878  76.32508

# momento de predicciones

In [ ]:
control_data_df.columns

Index(['raw_data', 'ecg_signals', 'info', 'ECG_Raw', 'ECG_Clean', 'ECG_Rate',
       'ECG_Quality', 'ECG_R_Peaks', 'ECG_P_Peaks', 'ECG_P_Onsets',
       'ECG_P_Offsets', 'ECG_Q_Peaks', 'ECG_R_Onsets', 'ECG_R_Offsets',
       'ECG_S_Peaks', 'ECG_T_Peaks', 'ECG_T_Onsets', 'ECG_T_Offsets',
       'ECG_Phase_Atrial', 'ECG_Phase_Completion_Atrial',
       'ECG_Phase_Ventricular', 'ECG_Phase_Completion_Ventricular', 'hrv_time',
       'HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_SDSD', 'HRV_CVNN',
       'HRV_CVSD', 'HRV_MedianNN', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN',
       'HRV_Prc20NN', 'HRV_Prc80NN', 'HRV_pNN50', 'HRV_pNN20', 'HRV_MinNN',
       'HRV_MaxNN', 'HRV_HTI', 'HRV_TINN', 'mean_quality'],
      dtype='object')

In [ ]:
drop_list=['HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_SDSD', 'HRV_CVNN',
       'HRV_CVSD', 'HRV_MedianNN', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN',
       'HRV_Prc20NN', 'HRV_Prc80NN', 'HRV_pNN50', 'HRV_pNN20', 'HRV_MinNN',
       'HRV_MaxNN', 'HRV_HTI', 'HRV_TINN', 'ECG_P_Peaks', 'ECG_Q_Peaks',
       'ECG_S_Peaks', 'ECG_T_Peaks', 'ECG_R_Peaks',
       'ratio p/r peaks', 'ratio q/r peaks', 'ratio s/r peaks',
       'ratio t/r peaks']
for item in control_data_df.columns:
    if item not in drop_list:
      control_data_df.drop(columns = [item], inplace = True)

control_data_df

,ECG_R_Peaks,ECG_P_Peaks,ECG_Q_Peaks,ECG_S_Peaks,ECG_T_Peaks,HRV_MeanNN,HRV_SDNN,HRV_RMSSD,HRV_SDSD,HRV_CVNN,...,HRV_pNN50,HRV_pNN20,HRV_MinNN,HRV_MaxNN,HRV_HTI,HRV_TINN,ratio p/r peaks,ratio q/r peaks,ratio s/r peaks,ratio t/r peaks
3600-7200,12,12,11,11,12,819.191919,26.365620,30.007715,30.772240,0.032185,...,0.000000,54.545455,786.111111,869.444444,3.666667,46.8750,1.0,0.916667,0.916667,1.000000
7200-10800,12,12,11,11,12,808.080808,27.398932,30.365571,30.694130,0.033906,...,9.090909,45.454545,772.222222,858.333333,5.500000,15.6250,1.0,0.916667,0.916667,1.000000
14400-18000,12,12,11,11,11,817.171717,20.253722,18.839806,19.545592,0.024785,...,0.000000,36.363636,780.555556,841.666667,3.666667,54.6875,1.0,0.916667,0.916667,0.916667
18000-21600,12,12,11,11,11,807.070707,26.039015,27.680385,29.140935,0.032264,...,9.090909,36.363636,780.555556,855.555556,3.666667,0.0000,1.0,0.916667,0.916667,0.916667
21600-25200,12,12,11,11,9,804.292929,23.483762,23.521070,24.499447,0.029198,...,0.000000,54.545455,775.000000,841.666667,3.666667,0.0000,1.0,0.916667,0.916667,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626400-630000,12,12,10,11,12,768.939394,91.184068,157.742563,166.191710,0.118584,...,27.272727,45.454545,527.777778,900.000000,11.000000,0.0000,1.0,0.833333,0.916667,1.000000
630000-633600,13,13,12,12,12,742.824074,18.290366,18.137924,19.006615,0.024623,...,0.000000,16.666667,716.666667,766.666667,4.000000,0.0000,1.0,0.923077,0.923077,0.923077
633600-637200,13,13,12,12,12,772.453704,35.974067,24.187121,23.715604,0.046571,...,8.333333,41.666667,708.333333,816.666667,6.000000,78.1250,1.0,0.923077,0.923077,0.923077
637200-640800,12,12,11,11,11,812.121212,23.807119,22.274245,22.810500,0.029315,...,0.000000,54.545455,772.222222,850.000000,3.666667,23.4375,1.0,0.916667,0.916667,0.916667


\

In [ ]:
data_ml = control_data_df
data_ml["ratio p/r peaks"] = data_ml["ECG_P_Peaks"].div(data_ml["ECG_R_Peaks"])
data_ml["ratio q/r peaks"] = data_ml["ECG_Q_Peaks"].div(data_ml["ECG_R_Peaks"])
data_ml["ratio s/r peaks"] = data_ml["ECG_S_Peaks"].div(data_ml["ECG_R_Peaks"])
data_ml["ratio t/r peaks"] = data_ml["ECG_T_Peaks"].div(data_ml["ECG_R_Peaks"])
data_ml


C:\Users\reegi\AppData\Local\Temp\ipykernel_16632\1647360774.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ml["ratio p/r peaks"] = data_ml["ECG_P_Peaks"].div(data_ml["ECG_R_Peaks"])
C:\Users\reegi\AppData\Local\Temp\ipykernel_16632\1647360774.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_ml["ratio q/r peaks"] = data_ml["ECG_Q_Peaks"].div(data_ml["ECG_R_Peaks"])
C:\Users\reegi\AppData\Local\Temp\ipykernel_16632\1647360774.py:4: SettingWithCopyWarning: 
A value is trying to be set on a c

,ECG_R_Peaks,ECG_P_Peaks,ECG_Q_Peaks,ECG_S_Peaks,ECG_T_Peaks,HRV_MeanNN,HRV_SDNN,HRV_RMSSD,HRV_SDSD,HRV_CVNN,...,HRV_pNN50,HRV_pNN20,HRV_MinNN,HRV_MaxNN,HRV_HTI,HRV_TINN,ratio p/r peaks,ratio q/r peaks,ratio s/r peaks,ratio t/r peaks
3600-7200,12,12,11,11,12,819.191919,26.365620,30.007715,30.772240,0.032185,...,0.000000,54.545455,786.111111,869.444444,3.666667,46.8750,1.0,0.916667,0.916667,1.000000
7200-10800,12,12,11,11,12,808.080808,27.398932,30.365571,30.694130,0.033906,...,9.090909,45.454545,772.222222,858.333333,5.500000,15.6250,1.0,0.916667,0.916667,1.000000
14400-18000,12,12,11,11,11,817.171717,20.253722,18.839806,19.545592,0.024785,...,0.000000,36.363636,780.555556,841.666667,3.666667,54.6875,1.0,0.916667,0.916667,0.916667
18000-21600,12,12,11,11,11,807.070707,26.039015,27.680385,29.140935,0.032264,...,9.090909,36.363636,780.555556,855.555556,3.666667,0.0000,1.0,0.916667,0.916667,0.916667
21600-25200,12,12,11,11,9,804.292929,23.483762,23.521070,24.499447,0.029198,...,0.000000,54.545455,775.000000,841.666667,3.666667,0.0000,1.0,0.916667,0.916667,0.750000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
626400-630000,12,12,10,11,12,768.939394,91.184068,157.742563,166.191710,0.118584,...,27.272727,45.454545,527.777778,900.000000,11.000000,0.0000,1.0,0.833333,0.916667,1.000000
630000-633600,13,13,12,12,12,742.824074,18.290366,18.137924,19.006615,0.024623,...,0.000000,16.666667,716.666667,766.666667,4.000000,0.0000,1.0,0.923077,0.923077,0.923077
633600-637200,13,13,12,12,12,772.453704,35.974067,24.187121,23.715604,0.046571,...,8.333333,41.666667,708.333333,816.666667,6.000000,78.1250,1.0,0.923077,0.923077,0.923077
637200-640800,12,12,11,11,11,812.121212,23.807119,22.274245,22.810500,0.029315,...,0.000000,54.545455,772.222222,850.000000,3.666667,23.4375,1.0,0.916667,0.916667,0.916667


In [ ]:
data_ml.columns

Index(['ECG_R_Peaks', 'ECG_P_Peaks', 'ECG_Q_Peaks', 'ECG_S_Peaks',
       'ECG_T_Peaks', 'HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_SDSD',
       'HRV_CVNN', 'HRV_CVSD', 'HRV_MedianNN', 'HRV_MadNN', 'HRV_MCVNN',
       'HRV_IQRNN', 'HRV_Prc20NN', 'HRV_Prc80NN', 'HRV_pNN50', 'HRV_pNN20',
       'HRV_MinNN', 'HRV_MaxNN', 'HRV_HTI', 'HRV_TINN', 'ratio p/r peaks',
       'ratio q/r peaks', 'ratio s/r peaks', 'ratio t/r peaks'],
      dtype='object')

In [ ]:
all_columns = ['HRV_MeanNN', 'HRV_SDNN', 'HRV_RMSSD', 'HRV_SDSD', 'HRV_CVNN',
       'HRV_CVSD', 'HRV_MedianNN', 'HRV_MadNN', 'HRV_MCVNN', 'HRV_IQRNN',
       'HRV_Prc20NN', 'HRV_Prc80NN', 'HRV_pNN50', 'HRV_pNN20', 'HRV_MinNN',
       'HRV_MaxNN', 'HRV_HTI', 'HRV_TINN', 'ECG_P_Peaks', 'ECG_Q_Peaks',
       'ECG_S_Peaks', 'ECG_T_Peaks', 'age', 'ECG_R_Peaks',
       'ratio p/r peaks', 'ratio q/r peaks', 'ratio s/r peaks',
       'ratio t/r peaks']
for item in data_ml:
  if item in all_columns:
    pass
  else:
    data_ml.drop(columns = [item], inplace = True)
data_ml

In [98]:
final_df = pd.DataFrame()
for item in drop_list:
  final_df[item]=data_ml[item]

In [99]:
final_df.drop(columns = "age", inplace = True)

KeyError: "['age'] not found in axis"

In [100]:
print(len(final_df))

123


In [101]:
import joblib
filename = r'C:\Users\reegi\Documents\TEC\10S\Proyecto Integrador\modelo_logreg.joblib'

loaded_model = joblib.load(filename)
result = loaded_model.predict(final_df).tolist()
print(result)

[1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1]


In [102]:
len(result)

123

In [103]:
prob = loaded_model.predict_proba(final_df)[:,1]

In [104]:
final_df["arrythmia"]= result

In [105]:
final_df["probability"] = prob

In [107]:
final_df[(final_df["probability"] < 0.7) & (final_df["arrythmia"] == 1)]["arrythmia"] = 0

C:\Users\reegi\AppData\Local\Temp\ipykernel_16632\1275358172.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  final_df[(final_df["probability"] < 0.7) & (final_df["arrythmia"] == 1)]["arrythmia"] = 0


In [96]:
(final_df["arrythmia"] == result)

3600-7200        True
7200-10800       True
14400-18000      True
18000-21600      True
21600-25200      True
                 ... 
626400-630000    True
630000-633600    True
633600-637200    True
637200-640800    True
644400-648000    True
Name: arrythmia, Length: 123, dtype: bool

In [97]:
final_df.to_csv("final.csv")

Dde aqui en adelante vamos a revisar que el df del jupyter y el de python sea exactamente el mismo.

In [ ]:
jupy_df = pd.read_csv("jupyter_result.csv")
py_df = pd.read_csv(r"C:\Users\reegi\Documents\TEC\10S\Proyecto Integrador\result_from_py.csv")


In [ ]:
py_df.columns == jupy_df.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [ ]:
py_df.sort_index()

,Unnamed: 0,HRV_MeanNN,HRV_SDNN,HRV_RMSSD,HRV_SDSD,HRV_CVNN,HRV_CVSD,HRV_MedianNN,HRV_MadNN,HRV_MCVNN,...,ECG_P_Peaks,ECG_Q_Peaks,ECG_S_Peaks,ECG_T_Peaks,ECG_R_Peaks,ratio p/r peaks,ratio q/r peaks,ratio s/r peaks,ratio t/r peaks,arrythmia
0,10000-20000,517.470588,121.378086,184.079636,189.199179,0.234560,0.355730,574.0,20.7564,0.036161,...,17,17,17,18,18,0.944444,0.944444,0.944444,1.000000,0
1,20000-30000,564.125000,71.438435,105.627017,109.331214,0.126636,0.187240,579.0,18.5325,0.032008,...,17,16,16,17,17,1.000000,0.941176,0.941176,1.000000,1
2,30000-40000,549.294118,92.303687,97.522754,100.720550,0.168041,0.177542,578.0,14.8260,0.025651,...,18,17,17,18,18,1.000000,0.944444,0.944444,1.000000,1
3,40000-50000,530.055556,103.683609,150.131511,153.797733,0.195609,0.283237,574.0,15.5673,0.027121,...,19,18,18,18,19,1.000000,0.947368,0.947368,0.947368,1
4,50000-60000,571.250000,13.684541,19.577198,20.250103,0.023955,0.034271,569.0,18.5325,0.032570,...,17,16,16,17,17,1.000000,0.941176,0.941176,1.000000,1
5,70000-80000,531.117647,107.765302,139.674174,144.247877,0.202903,0.262982,573.0,16.3086,0.028462,...,17,16,16,17,17,1.000000,0.941176,0.941176,1.000000,1
6,80000-90000,450.157895,143.230298,176.168007,180.631653,0.318178,0.391347,569.0,53.3736,0.093802,...,20,19,19,20,20,1.000000,0.950000,0.950000,1.000000,0
7,90000-100000,518.888889,119.407798,170.150107,175.358373,0.230122,0.327912,573.5,30.3933,0.052996,...,19,18,18,18,19,1.000000,0.947368,0.947368,0.947368,1
8,100000-110000,545.647059,91.478782,138.846588,143.400081,0.167652,0.254462,578.0,16.3086,0.028216,...,18,17,17,18,18,1.000000,0.944444,0.944444,1.000000,1
9,110000-120000,553.941176,66.187679,99.731515,102.999353,0.119485,0.180040,570.0,19.2738,0.033814,...,18,17,17,18,18,1.000000,0.944444,0.944444,1.000000,1
